In [3]:
import os


In [4]:
%pwd

'c:\\Users\\user\\Desktop\\Insurance E2E ML\\Insurance-Premium-End-to-End-\\research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'c:\\Users\\user\\Desktop\\Insurance E2E ML\\Insurance-Premium-End-to-End-'

In [7]:
#entity

from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionCOnfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

In [8]:
from src.inproject.constants import *
from src.inproject.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(self,
                 config_pathway = CONFIG_FILE_PATH,
                 param_pathway = PARAM_FILE_PATH,
                 schema_pathway = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_pathway)
        self.params = read_yaml(param_pathway)
        self.schema = read_yaml(schema_pathway)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionCOnfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionCOnfig(
            root_dir= config.root_dir,
            source_URL=config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [15]:
# data ingestion component

import os
import zipfile
import urllib.request as request
from src.inproject.logging import logger
from src.inproject.utils.common import read_yaml, get_size




In [16]:
#component

class DataIngestion:
    def __init__(self, config=DataIngestionCOnfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )

            logger.info(f'{filename} downloaded with the following info {headers}')

        else:
            logger.info(f'file already exist of size: {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        """
        zip_file_path: zip
        extract zip file into the directory
        function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
# pipeline

try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()


except Exception as e:
    raise e

[2025-01-01 17:34:08,085: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-01 17:34:08,089: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-01 17:34:08,089: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-01 17:34:08,095: INFO: common: created directory at: artifacts]
[2025-01-01 17:34:08,096: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-01 17:34:09,083: INFO: 2239081677: artifacts/data_ingestion/data.zip downloaded with the following info Content-Type: text/html; charset=utf-8
Date: Wed, 01 Jan 2025 17:34:08 GMT
Cache-Control: no-cache, no-store
Pragma: no-cache
Set-Cookie: ka_sessionid=7702ae77f552e5f874b832aa26d6424a; max-age=2626560; path=/
Set-Cookie: CSRF-TOKEN=CfDJ8L3o_FPsMQlKvPRAcaVvXWMW6D543bZNNvXC7NhgKg1C3kGa9E6ZNyCYb5TdT-1apKVNLjef9jakhdqKiC1B3oi8jz730pJzRG5P45kmag; path=/; secure; samesite=strict; httponly
Set-Cookie: XSRF-TOKEN=CfDJ8L3o_FPsMQlKvPRAcaVvXWNadPcKTp82fhDvEfLwU-CpuutH9j34vi

BadZipFile: File is not a zip file